In [11]:
import pandas as pd
import numpy as np


from langdetect import detect

In [12]:
# read training dataset
dataset_df = pd.read_csv('training_data.csv', names=['Text', 'Rating'], sep=';')

In [13]:
# check the shape of the dataset
dataset_df.shape

(40000, 2)

In [14]:
# show 10 data samples
dataset_df.head(10)

,Text,Rating
0,ร้านอาหารใหญ่มากกกกกกก \nเลี้ยวเข้ามาเจอห้องน้...,3
1,อาหารที่นี่เป็นอาหารจีนแคะที่หากินยากในบ้านเรา...,4
2,ปอเปี๊ยะสด ทุกวันนี้รู้สึกว่าหากินยาก (ร้านที่...,3
3,รัานคัพเค้กในเมืองไทยมีไม่มาก หลายๆคนอาจจะสงสั...,5
4,อร่อย!!! เดินผ่านDigital gatewayทุกวัน ไม่ยักร...,5
5,ร้านข้าวต้มกระดูกหมู ปากซอยพัฒนาการ 57 เป็นอีก...,4
6,วันนี้ได้มีโอกาสไปนั่งซดกาแฟที่ร้านวาวี แถวๆอา...,4
7,สารภาพว่าไม่เคยคิดจะไปต่อคิวซื้อมากินเองครับ บ...,3
8,เมื่อวันก่อนไปเดินเล่น แบบชิวๆๆ ที่สยามสแควร์แ...,5
9,วันก่อนไปเดินสยาม หลังจากห่างหายไป ประมาณ 6 เด...,5


## Exploratory analysis and Dataset Preparation

In [15]:
# Count distinct observations row-wise
dataset_df.nunique(axis=0)

Text      39993
Rating        5
dtype: int64

In [16]:
# Show rows that are duplicates (Mark duplicates as True except for the first occurrence.)
dataset_df[dataset_df.duplicated()]

,Text,Rating
9976,ร้านประจักษ์เป็ดย่าง เป็นร้านเป็ดย่างเปิดมานาน...,5
14129,ขนมที่เป็นสูตรเฉพาะของของทางร้าน ใส่ใจในเรื่อง...,5
14825,\nส่งท้ายเดือนแห่งความรัก ด้วยโปรโมชั่นเอาใจคน...,5
21038,ร้านอันดับหนึ่งในดวงใจของใครหลายๆคนรวมถึงตัวเร...,5
21040,ร้านอันดับหนึ่งในดวงใจของใครหลายๆคนรวมถึงตัวเร...,5
21048,สาขานี้พนักงานน่ารัก บริการดี ชอบโอโตยะตรงที่ม...,4
21049,สาขานี้พนักงานน่ารัก บริการดี ชอบโอโตยะตรงที่ม...,4


In [17]:
# Removes duplicate rows based on all columns)
dataset_df = dataset_df.drop_duplicates(ignore_index=True)

dataset_df.shape

(39993, 2)

In [18]:
# Drop rows with any column having NA/null data.
dataset_df = dataset_df.dropna()

In [19]:
dataset_df.shape

(39993, 2)

In [20]:
# check for class imbalance
dataset_df['Rating'].value_counts(sort=False)

1      415
2     1845
3    12171
4    18768
5     6794
Name: Rating, dtype: int64

Observation: Class 1 and class 2  are underrepresented. Suggested solution to mitigate this issue is to oversample the underrepresented classes.

### Check the language in reviews    

**Training Dataset.** language and corresponding number of reviews in this language  
{'th': 39723, 'en': 268, 'vi': 1, 'ja': 1}   

**Test Dataset**  
{'th': 6163, 'en': 39, 'ja': 1}    


Save reviews in thai and english separately   

In [21]:
all_reviews = np.array(dataset_df['Text'])

# save language and corresponding number of reviews in this language in a dictionary
lang_count = dict()

for review in all_reviews:
    # get the language of the current review
    lang_review = detect(review)
    
    # save to the dictionary counts
    if lang_review in lang_count: lang_count[lang_review] += 1
    else: lang_count[lang_review] = 1
        
# Note: language detection for almost 40k samples takes time!
print('Language and number of reviews in the language: ')
print(lang_count)

Language and number of reviews in the language: 
{'th': 39723, 'en': 268, 'vi': 1, 'ja': 1}


In [22]:
# Check out reviews language in test dataset
test_submit_df = pd.read_csv('test.csv', sep=';')

all_test_reviews = np.array(test_submit_df['review'])

# save language and corresponding number of reviews in this language in a dictionary
test_lang_count = dict()

for review in all_test_reviews:
    # get the language of the current review
    lang_review = detect(review)
    
    # save to the dictionary counts
    if lang_review in test_lang_count: test_lang_count[lang_review] += 1
    else: test_lang_count[lang_review] = 1

print('Language and number of reviews in the language: ')
print(lang_count)

Language and number of reviews in the language: 
{'th': 39723, 'en': 268, 'vi': 1, 'ja': 1}


In [23]:
# save Thai reviews and English reviews separately, non-Thai and non-English reviews - drop

# save only reviews in Thai language
dataset_th_df = dataset_df[dataset_df.apply(lambda row: detect(row['Text']) == 'th', axis=1)]

# save only reviews in English language
dataset_en_df = dataset_df[dataset_df.apply(lambda row: detect(row['Text']) == 'en', axis=1)]

In [24]:
# reset the index and avoid the old index being added as a column
dataset_en_df = dataset_en_df.reset_index(drop=True)
dataset_th_df = dataset_th_df.reset_index(drop=True)

# save datasets to files
dataset_en_df.to_csv('train_english_dataset.csv', index=False)
dataset_th_df.to_csv('train_thai_dataset.csv', index=False)